In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from googletrans import Translator, LANGUAGES

# 1- class translator sevice
class TranslationService:
    def __init__(self):
        self.translator = Translator() # ميثود الترجمه
    
    # ميثود ترجمة النصوص
    def translate_text(self, text, src_lang, dest_lang):
        translation = self.translator.translate(text, src=src_lang, dest=dest_lang)
        return translation.text # برجع النص اللي ترجمته
    

# 2- كلاس ادارة اللغات
class LanguageManager:
    def __init__(self):
        self.languages = LANGUAGES # لتخزين قائمة اللغات
    
    def get_language_code(self, lang_name): # للحصول علي رمز اللغه من خلال اسمها
        for code, name in self.languages.items():
            if name.lower() == lang_name.lower():
                return code
        return None
    
    def get_language_names(self): # عشان اجيب كل اسماء اللغات
        return list(self.languages.values())

#3- كلاس عشان احفظ فيه تاريخ كل الترجمه
class TranslationHistory:
    def __init__(self):
        self.history = [] # ليست عشان احفظ فيها التاريخ
    
    # ميثود عشان اضيف الترجمه للتاريخ
    def add_to_history(self, src_text, translated_text, src_lang, dest_lang):
        self.history.append((src_text, translated_text, src_lang, dest_lang)) # بضيف الترجمه لقائمة التاريخ
    
    def get_history(self):
        return self.history

# 4- كلاسس الواجهه
class UserInterface:
    def __init__(self, root, translation_service, language_manager, translation_history):
        self.root = root
        self.translation_service = translation_service
        self.language_manager = language_manager
        self.translation_history = translation_history

        self.setup_ui()# ميثود اعداد الواجهة

    def setup_ui(self):
        self.root.title("Translator Application")

        frame = ttk.Frame(self.root, padding="10") # اطار باستخدام ttk
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S)) # عشان يتمدد معايا مع التكبير والتصغير

        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        frame.columnconfigure(0, weight=1)
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(1, weight=1)

        self.src_lang_label = ttk.Label(frame, text="Source Language:")
        self.src_lang_label.grid(row=0, column=0, sticky=tk.W)

        # انشاء قائمة اللغات
        self.src_lang_combobox = ttk.Combobox(frame, values=self.language_manager.get_language_names(), width=30)
        self.src_lang_combobox.grid(row=0, column=1, sticky=(tk.W, tk.E))
        self.src_lang_combobox.current(21) # Defult To English

        self.dest_lang_label = ttk.Label(frame, text="Destination Language:")
        self.dest_lang_label.grid(row=1, column=0, sticky=tk.W)

        self.dest_lang_combobox = ttk.Combobox(frame, values=self.language_manager.get_language_names(), width=30)
        self.dest_lang_combobox.grid(row=1, column=1, sticky=(tk.W, tk.E))
        self.dest_lang_combobox.current(3) # Defult To Arabic

        self.text_label = ttk.Label(frame, text="Text to Translate:") # تسميه للنص اللي عاوز اترجمه
        self.text_label.grid(row=2, column=0, sticky=tk.W)

        self.text_entry = tk.Text(frame, height=10, width=50) # مربع نص الادخال
        self.text_entry.grid(row=3, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

        # انشاء زر الترجمه
        self.translate_button = ttk.Button(frame, text="Translate", command=self.translate_text)
        self.translate_button.grid(row=4, column=0, pady=10)

        # عشان اعرض تسمية نتيجة الترجمه
        self.result_label = ttk.Label(frame, text="Translation Result:")
        self.result_label.grid(row=5, column=0, sticky=tk.W)

        # مربع لعرض نتيجة الترجمه
        self.result_text = tk.Text(frame, height=10, width=50, state='disabled')
        self.result_text.grid(row=6, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

        self.history_button = ttk.Button(frame, text="Show History", command=self.show_history)
        self.history_button.grid(row=7, column=0, pady=10)

        for child in frame.winfo_children():
            child.grid_configure(padx=5, pady=5)
        
    # دالة ترجمة النص 
    def translate_text(self):
        src_lang = self.language_manager.get_language_code(self.src_lang_combobox.get())
        dest_lang = self.language_manager.get_language_code(self.dest_lang_combobox.get())
        text = self.text_entry.get("1.0", tk.END).strip()

        if src_lang and dest_lang and text:
            try:
                translated_text = self.translation_service.translate_text(text, src_lang, dest_lang)
                self.result_text.config(state='normal')
                self.result_text.delete("1.0", tk.END)
                self.result_text.insert(tk.END, translated_text)
                self.result_text.config(state='disabled')
                self.translation_history.add_to_history(text, translated_text, self.src_lang_combobox.get(), self.dest_lang_combobox.get())
            except Exception as e:
                messagebox.showerror("Error", str(e))
        else:
            messagebox.showwarning("Warning", "Please enter text and select languages.")
    
    # ميثود لعرض تاريخ الترجمات
    def show_history(self):
        history = self.translation_history.get_history()
        history_text = "\n\n".join([f"{src_text} ({src_lang}) -> {translated_text} ({dest_lang})" for src_text, translated_text, src_lang, dest_lang in history])
        if not history_text:
            history_text = "No translation history yet."
        messagebox.showinfo("Translation History", history_text)

# 5- كلاس التطبيق الرئيسي
class TranslatorApp:
    def __init__(self, root):
        self.translation_service = TranslationService()
        self.language_manager = LanguageManager()
        self.translation_history = TranslationHistory()
        self.ui = UserInterface(root, self.translation_service, self.language_manager, self.translation_history)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslatorApp(root)
    root.mainloop()






